In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect('../../datasets/checking-logs.sqlite')

#### Get the schema of the table test

In [3]:
print(pd.read_sql("PRAGMA table_info(test)", con))

   cid             name       type  notnull dflt_value  pk
0    0              uid       TEXT        0       None   0
1    1          labname       TEXT        0       None   0
2    2  first_commit_ts  TIMESTAMP        0       None   0
3    3    first_view_ts  TIMESTAMP        0       None   0


#### Get only the first 10 rows of the table test to check what the table looks like

In [4]:
sql = """
SELECT *
FROM test
limit 10
"""
print(pd.read_sql(sql, con))


       uid   labname             first_commit_ts               first_view_ts
0  user_17  project1  2020-04-18 07:56:45.408648  2020-04-18 10:56:55.833899
1  user_30    laba04  2020-04-18 13:36:53.971502  2020-04-17 22:46:26.785035
2  user_30   laba04s  2020-04-18 14:51:37.498399  2020-04-17 22:46:26.785035
3  user_14    laba04  2020-04-18 15:14:00.312338  2020-04-18 10:53:52.623447
4  user_14   laba04s  2020-04-18 22:30:30.247628  2020-04-18 10:53:52.623447
5  user_19    laba04  2020-04-20 19:05:01.297780  2020-04-21 20:30:38.034966
6  user_25    laba04  2020-04-20 19:16:50.673054  2020-05-09 23:54:54.260791
7  user_21    laba04  2020-04-21 17:48:00.487806  2020-04-22 22:40:36.824081
8  user_30  project1  2020-04-22 12:36:24.053518  2020-04-17 22:46:26.785035
9  user_21   laba04s  2020-04-22 20:09:21.857747  2020-04-22 22:40:36.824081


#### Find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query

##### Do this by joining the table with the table deadlines

In [5]:
sql = """
SELECT uid, MIN((strftime('%s', first_commit_ts) - d.deadlines)/3600) as df_min
FROM test t
LEFT JOIN deadlines d on t.labname = d.labs
WHERE labname != 'project1'
"""

df_min = pd.read_sql(sql, con)
print(df_min)

       uid  df_min
0  user_30    -202


##### Do the same thing, but for the maximum, using only one query, the dataframe name is df_max

In [6]:
sql = """
SELECT uid, MAX((strftime('%s', first_commit_ts) - d.deadlines)/3600) as df_max
FROM test t
LEFT JOIN deadlines d on t.labname = d.labs
WHERE labname != 'project1'
"""

df_max = pd.read_sql(sql, con)
print(df_max)

       uid  df_max
0  user_25      -2


#### Do the same thing but for the average, using only one query, this time your dataframe should not include the uid column, and the dataframe name is df_avg

In [7]:
sql = """
SELECT avg((strftime('%s', first_commit_ts) - d.deadlines)/3600) as df_avg
FROM test t
LEFT JOIN deadlines d on t.labname = d.labs
WHERE labname != 'project1'
"""

df_avg = pd.read_sql(sql, con)
print(df_avg)

   df_avg
0 -89.125


#### You need to calculate the correlation coefficient between the number of pageviews and the difference

In [8]:
sql = """
WITH user_data AS (
    SELECT 
        t.uid,
        AVG((strftime('%s', t.first_commit_ts) - d.deadlines)/3600.0) AS avg_diff,
        COUNT(datetime) AS pageviews
    FROM test t
    LEFT JOIN deadlines d ON t.labname = d.labs
    LEFT JOIN pageviews p ON p.uid = t.uid
    WHERE t.labname != 'project1'
    GROUP BY t.uid
)
SELECT 
    uid,
    avg_diff,
    pageviews
FROM user_data 
"""
views_diff = pd.read_sql(sql, con)
print(views_diff)
correlation = views_diff[['pageviews', 'avg_diff']].corr().iloc[0, 1]
print(f"{correlation:.4f}")

        uid    avg_diff  pageviews
0    user_1  -65.119778        140
1   user_10  -75.242444        445
2   user_14 -159.568796        429
3   user_17  -62.207667        235
4   user_18   -6.368148          9
5   user_19  -99.440417         64
6   user_21  -96.111181         40
7   user_25  -93.474944        895
8   user_28  -86.793833        745
9    user_3 -105.738222       1585
10  user_30 -145.528681         12
-0.1850


In [9]:
con.close()